# exercise

Our final output is a list of Customers containing their data and their coordinates. It can be represented by list of dicts.  
We will get our Customers from XML exercise and their coordinates from `googleapis` (JSON exercise).

---
1. Fetch customers from **http://www.thomas-bayer.com/sqlrest/CUSTOMER/**. Create list named **customers** containing dicts with Customers `id` and `link`.  
    Output should look like this: 
           [{'id': '2', 'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/2/'},
            {'id': '3', 'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/3/'},
            ... 
           ]
           
2. For each Customer fetch their details using their links. Create list named **customers_data** containing dicts with Customers `id`, `link`, `firstname`, `lastname`, `city` and `street`.  
    Output should look like this:
        [{'city': 'Lyon',
          'firstname': 'Anne',
          'id': '2',
          'lastname': 'Miller',
          'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/2/',
          'street': '20 Upland Pl.'},
         {'city': 'San Francisco',
          'firstname': 'Michael',
          'id': '3',
          'lastname': 'Clancy',
          'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/3/',
          'street': '542 Upland Pl.'},
          ...
         ]
         
3. For each Customer fetch their coordinates using **http://maps.googleapis.com/maps/api/geocode/json** and Customers data (`"{street}, {city}"`).  
    Create list named **customers_data_geo** containing dicts with Customers `id`, `link`, `firstname`, `lastname`, `city`, `street`, `lat` and `lng`.  
    If Customers location is not found, then fill `lat` and `lng` with `None`, otherwise use first found location.  
    Output should look like this:
        [{'city': 'Lyon',
          'firstname': 'Anne',
          'id': '2',
          'lastname': 'Miller',
          'lat': None,
          'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/2/',
          'lng': None,
          'street': '20 Upland Pl.'},
         {'city': 'San Francisco',
          'firstname': 'Michael',
          'id': '3',
          'lastname': 'Clancy',
          'lat': 37.7302685,
          'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/3/',
          'lng': -122.4659227,
          'street': '542 Upland Pl.'},
          ...
         ]

In [33]:
import requests
import xml.etree.ElementTree as ET

response = requests.get('http://www.thomas-bayer.com/sqlrest/CUSTOMER/')
customers_xml = ET.fromstring(response.text)

customers = customers_xml.getchildren()

customer_temp = []

for customer in customers:
    d = {'id': customer.text,
        'link': customer.get('{http://www.w3.org/1999/xlink}href')}
    
    customer_temp.append(d)


In [2]:
customers = customer_temp

In [17]:
customers_data = []

for customer in customers: 
    response = requests.get(customer['link'])
    customer_xml = ET.fromstring(response.text)
    firstname = customer_xml.find('FIRSTNAME').text
    lastname = customer_xml.find('LASTNAME').text
    street = customer_xml.find('STREET').text
    city = customer_xml.find('CITY').text
    d = {'id': customer['id'],
        'link': customer['link'],
        'firstname': firstname,
        'lastname': lastname,
        'street': street,
        'city': city} 
    customers_data.append(d)
    
customers_data

[{'city': 'Dallas',
  'firstname': 'Laura',
  'id': '0',
  'lastname': 'Steel',
  'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/0/',
  'street': '429 Seventh Av.'},
 {'city': 'Olten',
  'firstname': 'Susanne',
  'id': '1',
  'lastname': 'King',
  'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/1/',
  'street': '366 - 20th Ave.'},
 {'city': 'Lyon',
  'firstname': 'Anne',
  'id': '2',
  'lastname': 'Miller',
  'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/2/',
  'street': '20 Upland Pl.'},
 {'city': 'San Francisco',
  'firstname': 'Michael',
  'id': '3',
  'lastname': 'Clancy',
  'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/3/',
  'street': '542 Upland Pl.'},
 {'city': 'Dallas',
  'firstname': 'Sylvia',
  'id': '4',
  'lastname': 'Ringer',
  'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/4/',
  'street': '365 College Av.'},
 {'city': 'Paris',
  'firstname': 'Laura',
  'id': '5',
  'lastname': 'Miller',
  'link': 'http://www.thomas-bayer.com/sqlrest/C

In [53]:
base_url = 'http://maps.googleapis.com/maps/api/geocode/json'

cust_temp = customers_data[0]
customer_params = {'address': cust_temp['street'] + ', ' + cust_temp['city']}

print(cust_temp['city'] + ' - ' + cust_temp['street'])

response = requests.get(base_url, params = customer_params)


#response.status_code
#customer_params
results = response.json()['results']
#len(results)
cust_geo = results[0]['geometry']['location']
d = {'lat': cust_geo['lat'],
    'lng': cust_geo['lng']}

d

Dallas - 429 Seventh Av.


{'lat': 32.7529735, 'lng': -96.817878}

In [129]:
base_url = 'http://maps.googleapis.com/maps/api/geocode/json'
customers_data_geo = []

for customer in customers_data:
    customer_params = {'address': customer['street'] + ', ' + customer['city']}
    response = requests.get(base_url, params = customer_params)
    results = response.json()['results']
    #print(results)
    for result in results:

        lng = result['geometry']['location']['lng']
        lat = result['geometry']['location']['lat']


        #lat = result['geometry']
#        lng = result['location']['lng']
#        lat = result['location']['lat']
        print('Logitude {:>15.10}    Latitude {:>15.15}'.format(lng,lat))
        #print(lng, lat)
        #customers_data_geo[customer].append(d)

    #cust_geo = results[customer][0]['geometry']['location']
    #d = {'lat': cust_geo['lat'],'lng': cust_geo['lng']}
    
#customers_data_geo

Logitude      -96.817878    Latitude      32.7529735
Logitude    -122.4659227    Latitude      37.7302685
Logitude     -81.1747597    Latitude      35.3111223
Logitude     -81.1743511    Latitude      35.3183272
Logitude     -96.4733975    Latitude      32.7408249
Logitude     -97.3346787    Latitude      32.7411058
Logitude      -97.249516    Latitude       32.938978
Logitude    -122.4236687    Latitude      37.7336686
Logitude     -95.5630321    Latitude      33.6589008
Logitude     -95.5629968    Latitude        33.66274
Logitude     -84.2551704    Latitude      38.2120812
Logitude     -84.1465062    Latitude      38.3007949
Logitude     -84.2556487    Latitude      38.2104528
Logitude     -78.5493437    Latitude      40.1024231
Logitude    -122.4638097    Latitude      37.7293284
Logitude    -122.3063479    Latitude      47.6043002
Logitude     -82.6056312    Latitude      39.7200114
Logitude     -82.5959863    Latitude       39.719752
Logitude     -82.5957017    Latitude      39.7

In [135]:
base_url = 'http://maps.googleapis.com/maps/api/geocode/json'
customers_data_geo = []

for customer in customers_data:
    customer_params = {'address': customer['street'] + ', ' + customer['city']}
    response = requests.get(base_url, params = customer_params)
    results = response.json()['results']
    customers_data_geo.append(customer)
    for result in results:
        lng = result['geometry']['location']['lng']
        lat = result['geometry']['location']['lat']
        #d = {'longitude': lng,
        #   'lattitude': lat}
        customers_data_geo[longitude] = lng
        #, lattitude': lat)
        

        #print(lng, lat)


    #cust_geo = results[customer][0]['geometry']['location']
    #d = {'lat': cust_geo['lat'],'lng': cust_geo['lng']}
    
customers_data_geo



NameError: name 'longitude' is not defined